# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [26]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import json


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'classDB', 'config', 'fruitsDB', 'local', 'met']

In [44]:
# assign the uk_food database to a variable name
database_name = "uk_food"
collection_name = "establishments"
db = mongo[database_name]
establishments = db[collection_name]

In [27]:
with open("/Users/go2themax99/Downloads/Starter_Code 8/Resources/establishments.json") as file:
    data = json.load(file)

In [45]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [46]:
# review a document in the establishments collection
db[collection_name].find_one()

{'_id': ObjectId('646291719d58a79b795aaf57'),
 'name': 'Penang Flavours',
 'location': 'Greenwich',
 'cuisine': 'Halal',
 'rating': None,
 'reviews': []}

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [47]:
# Create a dictionary for the new restaurant data
restaurant_data = {
    "name": "Penang Flavours",
    "location": "Greenwich",
    "cuisine": "Halal",
    "rating": None,
    "reviews": []
}

In [48]:
# Insert the new restaurant into the collection
result = establishments.insert_one(restaurant_data)

In [49]:
# Check that the new restaurant was inserted
print("Inserted document ID:", result.inserted_id)

Inserted document ID: 646296659d58a79b795aaf59


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [50]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}

# Find the document matching the query and return only the specified fields
result = establishments.find_one(query, {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})

# Print the result
pprint(result)

None


3. Update the new restaurant with the `BusinessTypeID` you found.

In [42]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": "YOUR_BUSINESS_TYPE_ID"}}
)

In [43]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}

result = establishments.find_one(query)
print(result)

None


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [51]:
# Find how many documents have LocalAuthorityName as "Dover"
count = establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the count
print(count)

0


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [ ]:
# Check if any remaining documents include Dover
remaining_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

print("Remaining count:", remaining_count)

In [52]:
# Check that other documents remain with 'find_one'
db[collection_name].find_one()

{'_id': ObjectId('646291719d58a79b795aaf57'),
 'name': 'Penang Flavours',
 'location': 'Greenwich',
 'cuisine': 'Halal',
 'rating': None,
 'reviews': []}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
update_query = [
    {"$set": {"longitude": {"$toDouble": "$longitude"}}},
    {"$set": {"latitude": {"$toDouble": "$latitude"}}}
]

establishments.update_many({}, update_query)

Use `update_many` to convert `RatingValue` to integer numbers.

In [53]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
update_int_query = [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
]

establishments.update_many({}, update_int_query)


In [ ]:
# Check that the coordinates and rating value are now numbers
query = {"BusinessName": "Penang Flavours"}

result = establishments.find_one(query)
print(result)